# Data preparation

In [1]:
%load_ext autoreload
%autoreload 2

from qdrant_client import QdrantClient, models
import os
import sys

# Do this to enable importing modules
src_path = os.path.join(os.path.abspath(""), "..")
sys.path.insert(0, src_path)

## MongoDb data

In [10]:
CLEAN_MONGODB = True

if CLEAN_MONGODB:
    from pymongo import MongoClient
    host = "mongodb://localhost:30001,localhost:30002,localhost:30003/?replicaSet=my-replica-set"
    mongo_client = MongoClient(host)

    db = mongo_client["crypto-articles"]
    db["articles"].drop()

2024-06-04 17:29:17,016 - INFO - {"message": "Waiting for suitable server to become available", "selector": "<function writable_server_selector at 0x17ff21e10>", "operation": "drop", "topologyDescription": "<TopologyDescription id: 665f32cdbf60523316ad4cd8, topology_type: ReplicaSetNoPrimary, servers: [<ServerDescription ('localhost', 30001) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30002) server_type: Unknown, rtt: None>, <ServerDescription ('localhost', 30003) server_type: Unknown, rtt: None>]>", "clientId": {"$oid": "665f32cdbf60523316ad4cd8"}, "remainingTimeMS": 29}


## Qdrant Data

In [11]:
from finetuning.settings import settings
from qdrant_client.http.models import Batch, Distance, VectorParams

EMBEDDING_SIZE=384
CLEAN_COLLECTION = False

_client = QdrantClient(
    host=settings.QDRANT_DATABASE_HOST,
    port=settings.QDRANT_DATABASE_PORT,
)


if CLEAN_COLLECTION:
    _client.delete_collection(collection_name="cleaned_articles")
    _client.delete_collection(collection_name="vector_articles")

    _client.create_collection(collection_name="cleaned_articles", vectors_config={})
    _client.create_collection(collection_name="vector_articles", vectors_config=VectorParams(size=EMBEDDING_SIZE, distance=Distance.COSINE))

_client.get_collections()

2024-06-04 18:20:02,938 - INFO - HTTP Request: GET http://localhost:6333/collections "HTTP/1.1 200 OK"


CollectionsResponse(collections=[CollectionDescription(name='cleaned_articles'), CollectionDescription(name='vector_articles')])

In [31]:
import textwrap

i = 2

# for article in articles:
#     wrapped_string = textwrap.fill(article["cleaned_content"], width=100)
#     print("\n")
#     print(article["source"])
#     print(wrapped_string)

In [5]:
from finetuning.utils.openai_helper import OpenAIHandler
from finetuning.utils.data_formatter import DataFormatter
from finetuning.data_generator import DatasetGenerator

2024-06-04 17:09:14,137 - INFO - trying to connect to qdrant
2024-06-04 17:09:14,138 - INFO - host
2024-06-04 17:09:14,138 - INFO - localhost
2024-06-04 17:09:14,138 - INFO - port
2024-06-04 17:09:14,138 - INFO - 6333


In [21]:
collection_name = "cleaned_articles"
openai_handler = OpenAIHandler()
formatter = DataFormatter()
generator = DatasetGenerator(openai_handler, formatter)
all_contents = generator.fetch_all_cleaned_content(collection_name)
training_data = generator.generate_training_data(all_contents[:10], 1)
training_data

In [7]:
generator.push_to_comet(training_data, collection_name)

2024-06-04 17:14:31,782 - INFO - Starting to push data to Comet: cleaned_articles
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/cancamilo/crypto-reporter/49e2cc8a14ec48d789aa7b2e46374f10

2024-06-04 17:14:35,687 - INFO - Writing data to file: cleaned_articles.json
2024-06-04 17:14:35,688 - INFO - Data written to file successfully
2024-06-04 17:14:35,689 - INFO - Artifact created and file added: cleaned_articles.json
COMET INFO: Artifact 'cleaned_articles' version 2.0.0 created (previous was: 1.0.0)
COMET INFO: Scheduling the upload of 1 assets for a size of 14.42 KB, this can take some time
COMET INFO: Artifact 'cancamilo/cleaned_articles:2.0.0' has started uploading asynchronously
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Come

In [61]:
cleaned_articles = generator.fetch_all_cleaned_content("cleaned_articles")
len(cleaned_articles)

2024-06-04 17:00:12,985 - INFO - HTTP Request: POST http://localhost:6333/collections/cleaned_articles/points/scroll "HTTP/1.1 200 OK"


30

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser

all_contents = cleaned_articles[140:150]
batch_size = 1


openai_handler = OpenAIHandler()
formatter = DataFormatter()

response = []

for i in range(0, len(all_contents), batch_size):
    batch = all_contents[i : i + batch_size]
    initial_prompt = formatter.format_prompt(batch, i)
    print(initial_prompt)
    batch_result = openai_handler.request(initial_prompt)

    # only process batch if response is valid
    if len(batch_result) > 0:
        response += batch_result
        for j in range(i, i + batch_size):
            response[j]["content"] = all_contents[j]

# self.push_to_comet(response, collection_name)
response


In [19]:
import pandas as pd

df = pd.DataFrame(cleaned_articles, columns=["content"])
df["length"] = df["content"].apply(lambda x: len(x.split(" ")))

,length
count,323.000000
mean,175.275542
std,98.662932
min,13.000000
25%,112.000000
50%,169.000000
75%,219.000000
max,1023.000000


In [27]:
df["content"].isna().sum()

0